In [ ]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import nibabel as nib
from PyQt5 import QtWidgets, QtGui
import sys


class ColorPickerApp(QtWidgets.QWidget):
    def __init__(self, labels_out, render_window, lut, renderer):
        super().__init__()
        self.labels_out = labels_out
        self.render_window = render_window
        self.lut = lut
        self.renderer = renderer
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Color Picker for Connected Components")
        self.setGeometry(100, 100, 400, 600)

        layout = QtWidgets.QVBoxLayout()

        # Instructions
        instructions = QtWidgets.QLabel("Select a label and assign a color:")
        instructions.setFont(QtGui.QFont("Arial", 12))
        layout.addWidget(instructions)

        # List of labels
        self.label_list = QtWidgets.QListWidget()
        for i in range(1, np.max(self.labels_out) + 1):
            self.label_list.addItem(f"Component {i}")
        layout.addWidget(self.label_list)

        # Color picker button
        self.color_button = QtWidgets.QPushButton("Choose Color")
        self.color_button.clicked.connect(self.openColorPicker)
        layout.addWidget(self.color_button)

        # Apply button
        self.apply_button = QtWidgets.QPushButton("Apply Changes")
        self.apply_button.clicked.connect(self.applyChanges)
        layout.addWidget(self.apply_button)

        # Set specific views buttons
        self.view1_button = QtWidgets.QPushButton("View 1: Top View")
        self.view1_button.clicked.connect(self.setView1)
        layout.addWidget(self.view1_button)

        self.view2_button = QtWidgets.QPushButton("View 2: Side View")
        self.view2_button.clicked.connect(self.setView2)
        layout.addWidget(self.view2_button)

        self.view3_button = QtWidgets.QPushButton("View 3: Front View")
        self.view3_button.clicked.connect(self.setView3)
        layout.addWidget(self.view3_button)

        self.view4_button = QtWidgets.QPushButton("View 4: Isometric View")
        self.view4_button.clicked.connect(self.setView4)
        layout.addWidget(self.view4_button)

        # Render button
        self.render_button = QtWidgets.QPushButton("Render")
        self.render_button.clicked.connect(self.render)
        layout.addWidget(self.render_button)

        self.setLayout(layout)

    def openColorPicker(self):
        selected_items = self.label_list.selectedItems()
        if not selected_items:
            QtWidgets.QMessageBox.warning(self, "Warning", "Please select a component first!")
            return

        # Keep the color picker open for multiple selections
        selected_label = int(selected_items[0].text().split()[-1])  # Extract label index

        # Open color picker dialog
        color_dialog = QtWidgets.QColorDialog(self)
        color_dialog.setOption(QtWidgets.QColorDialog.DontUseNativeDialog, True)
        color_dialog.setWindowTitle(f"Choose Color for Component {selected_label}")

        def apply_color():
            color = color_dialog.currentColor()
            if color.isValid():
                self.lut.SetTableValue(
                    selected_label,
                    color.redF(),
                    color.greenF(),
                    color.blueF(),
                    1.0,
                )
                self.render_window.Render()  # Update the render window with the new color

        # Connect the 'Ok' button of the color dialog to apply the color
        color_dialog.currentColorChanged.connect(lambda color: apply_color())

        # Show the color picker dialog
        color_dialog.exec_()

    def applyChanges(self):
        # Update the lookup table in the mapper
        self.render_window.Render()

    def setView1(self):
        # Top view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(0, 0, 1)
        camera.SetViewUp(0, 1, 0)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView2(self):
        # Side view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(1, 0, 0)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView3(self):
        # Front view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(0, -1, 0)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView4(self):
        # Isometric view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(1, 1, 1)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def render(self):
        # Start the rendering process
        self.render_window.Render()


def main():
    # Load your segmentation data
    nii_path = "isotropic_patient001_frame12_gt.nii.gz"
    img = nib.load(nii_path)
    labels_in = img.get_fdata()

    # Ensure that labels are consecutive integers starting from 1
    labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

    # Create a VTK image data object
    vtk_image = vtk.vtkImageData()
    vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
    vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

    # Copy NumPy array data to VTK image
    vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
    vtk_image.GetPointData().SetScalars(vtk_array)

    # Create a VTK Discrete Marching Cubes algorithm
    discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
    discrete_marching_cubes.SetInputData(vtk_image)
    discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

    # Apply smoothing
    smooth_filter = vtk.vtkSmoothPolyDataFilter()
    smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
    smooth_filter.SetNumberOfIterations(700)

    # Create a VTK renderer and render window
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.SetWindowName("Connected Components 3D Visualization")
    render_window.SetSize(800, 800)
    render_window.AddRenderer(renderer)

    # Create a VTK render window interactor
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Create a VTK lookup table
    lut = vtk.vtkLookupTable()
    lut.SetNumberOfTableValues(np.max(labels_out) + 1)
    lut.Build()

    # Assign initial random colors to components
    for i in range(1, np.max(labels_out) + 1):
        color = np.random.rand(3)  # Random RGB color
        lut.SetTableValue(i, color[0], color[1], color[2])

    # Create a VTK PolyDataMapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
    mapper.SetLookupTable(lut)
    mapper.SetScalarRange(0, np.max(labels_out))

    # Create a VTK actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(1, 1, 1)  # White background
    renderer.ResetCamera()

    # Create the PyQt5 app
    app = QtWidgets.QApplication(sys.argv)
    window = ColorPickerApp(labels_out, render_window, lut, renderer)
    window.show()

    # Start the interactor
    render_window_interactor.Initialize()
    render_window_interactor.Start()

    sys.exit(app.exec_())


if __name__ == "__main__":
    main()


In [ ]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import nibabel as nib
from PyQt5 import QtWidgets, QtGui
import sys


class ColorPickerApp(QtWidgets.QWidget):
    def __init__(self, labels_out, render_window, lut, renderer):
        super().__init__()
        self.labels_out = labels_out
        self.render_window = render_window
        self.lut = lut
        self.renderer = renderer
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Color Picker for Connected Components")
        self.setGeometry(100, 100, 400, 600)

        layout = QtWidgets.QVBoxLayout()

        # Instructions
        instructions = QtWidgets.QLabel("Select a label and assign a color:")
        instructions.setFont(QtGui.QFont("Arial", 12))
        layout.addWidget(instructions)

        # List of labels
        self.label_list = QtWidgets.QListWidget()
        for i in range(1, np.max(self.labels_out) + 1):
            self.label_list.addItem(f"Component {i}")
        layout.addWidget(self.label_list)

        # Color picker button
        self.color_button = QtWidgets.QPushButton("Choose Color")
        self.color_button.clicked.connect(self.openColorPicker)
        layout.addWidget(self.color_button)

        # Apply button
        self.apply_button = QtWidgets.QPushButton("Apply Changes")
        self.apply_button.clicked.connect(self.applyChanges)
        layout.addWidget(self.apply_button)

        # Set specific views buttons
        self.view1_button = QtWidgets.QPushButton("View 1: Top View")
        self.view1_button.clicked.connect(self.setView1)
        layout.addWidget(self.view1_button)

        self.view2_button = QtWidgets.QPushButton("View 2: Side View")
        self.view2_button.clicked.connect(self.setView2)
        layout.addWidget(self.view2_button)

        self.view3_button = QtWidgets.QPushButton("View 3: Front View")
        self.view3_button.clicked.connect(self.setView3)
        layout.addWidget(self.view3_button)

        self.view4_button = QtWidgets.QPushButton("View 4: Isometric View")
        self.view4_button.clicked.connect(self.setView4)
        layout.addWidget(self.view4_button)

        # Render button
        self.render_button = QtWidgets.QPushButton("Render")
        self.render_button.clicked.connect(self.render)
        layout.addWidget(self.render_button)

        self.setLayout(layout)

    def openColorPicker(self):
        selected_items = self.label_list.selectedItems()
        if not selected_items:
            QtWidgets.QMessageBox.warning(self, "Warning", "Please select a component first!")
            return

        # Keep the color picker open for multiple selections
        selected_label = int(selected_items[0].text().split()[-1])  # Extract label index

        # Open color picker dialog
        color_dialog = QtWidgets.QColorDialog(self)
        color_dialog.setOption(QtWidgets.QColorDialog.DontUseNativeDialog, True)
        color_dialog.setWindowTitle(f"Choose Color for Component {selected_label}")

        def apply_color():
            color = color_dialog.currentColor()
            if color.isValid():
                self.lut.SetTableValue(
                    selected_label,
                    color.redF(),
                    color.greenF(),
                    color.blueF(),
                    1.0,
                )
                self.render_window.Render()  # Update the render window with the new color

        # Connect the 'Ok' button of the color dialog to apply the color
        color_dialog.currentColorChanged.connect(lambda color: apply_color())

        # Show the color picker dialog
        color_dialog.exec_()

    def applyChanges(self):
        # Update the lookup table in the mapper
        self.render_window.Render()

    def setView1(self):
        # Top view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(0, 0, 1)
        camera.SetViewUp(0, 1, 0)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView2(self):
        # Side view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(1, 0, 0)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView3(self):
        # Front view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(0, -1, 0)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def setView4(self):
        # Isometric view
        camera = self.renderer.GetActiveCamera()
        camera.SetPosition(1, 1, 1)
        camera.SetViewUp(0, 0, 1)
        camera.SetFocalPoint(0, 0, 0)
        self.renderer.ResetCamera()
        self.render_window.Render()

    def render(self):
        # Start the rendering process
        self.render_window.Render()


def main():
    # Load your segmentation data
    nii_path = "468_day2_f_5.nii.gz"
    img = nib.load(nii_path)
    labels_in = img.get_fdata()

    # Ensure that labels are consecutive integers starting from 1
    labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

    # Create a VTK image data object
    vtk_image = vtk.vtkImageData()
    vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
    vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

    # Copy NumPy array data to VTK image
    vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
    vtk_image.GetPointData().SetScalars(vtk_array)

    # Create a VTK Discrete Marching Cubes algorithm
    discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
    discrete_marching_cubes.SetInputData(vtk_image)
    discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

    # Apply smoothing
    smooth_filter = vtk.vtkSmoothPolyDataFilter()
    smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
    smooth_filter.SetNumberOfIterations(700)

    # Create a VTK renderer and render window
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.SetWindowName("Connected Components 3D Visualization")
    render_window.SetSize(800, 800)
    render_window.AddRenderer(renderer)

    # Create a VTK render window interactor
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Create a VTK lookup table
    lut = vtk.vtkLookupTable()
    lut.SetNumberOfTableValues(np.max(labels_out) + 1)
    lut.Build()

    # Assign initial random colors to components
    for i in range(1, np.max(labels_out) + 1):
        color = np.random.rand(3)  # Random RGB color
        lut.SetTableValue(i, color[0], color[1], color[2])

    # Create a VTK PolyDataMapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
    mapper.SetLookupTable(lut)
    mapper.SetScalarRange(0, np.max(labels_out))

    # Create a VTK actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(1, 1, 1)  # White background
    renderer.ResetCamera()

    # Create the PyQt5 app
    app = QtWidgets.QApplication(sys.argv)
    window = ColorPickerApp(labels_out, render_window, lut, renderer)
    window.show()

    # Start the interactor
    render_window_interactor.Initialize()
    render_window_interactor.Start()

    sys.exit(app.exec_())


if __name__ == "__main__":
    main()
